In [1]:
import os
import numpy as np
import DataPreprocessing as DP
from matplotlib import pyplot as plt
from astropy.io import fits
import tensorflow as tf
from keras.models import Sequential
#from keras.layers import LSTM, Dense, Embedding, Input, Flatten
from keras import optimizers
from keras.models import Model

from keras.layers import Dense, Dropout, LSTM, Flatten, Input
from keras.optimizers import Adam


import time
Tstart = time.time()

In [2]:
#os.environ["CUDA_VISIBLE_DEVICES"]="0" #for GPU usagegpus = tf.config.experimental.list_physical_devices(device_type='XLA_GPU')
gpus = tf.config.experimental.list_physical_devices(device_type='XLA_GPU')
print(gpus)
tf.config.experimental.set_visible_devices(devices=gpus, device_type='XLA_GPU')

[PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU')]


In [3]:
inputd = Input(shape=(1214,4552))#(1214,4552)
inputd.shape

TensorShape([None, 1214, 4552])

In [17]:
#LSTM net
x = LSTM(128,activation='tanh')(inputd)#(1214,4552)
x = Dropout(0.16)(x)#()

#hidden net
encoded = Dense(2, activation='relu', name='embedding')(x)
x.shape
x = Reshape((256,8))(x)

#decoded = Dense(1,activation='softmax')(encoded)

TensorShape([None, 128])

In [15]:
autoencoder = Model(inputd, x)
#autoencoder = multi_gpu_model(input_spe, decoded)
optimizer_adam = optimizers.Adam(lr=0.001)
autoencoder.compile(optimizer=optimizer_adam, loss='categorical_crossentropy', metrics=['accuracy'])####loss function
#autoencoder.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(autoencoder.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1214, 4552)]      0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               2396672   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
Total params: 2,396,672
Trainable params: 2,396,672
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
#Dataset
#norm spectrum
kinds = ['boss_cv','boss_da+ms','boss_db','boss_db+ms','boss_dq','boss_dz','fgkm','hotstars','wd','wdsb2','yso','hotstars_m']
flux_cv, spectrum_cv = DP.Preprocessing7('/home/njl/ML/optical/'+kinds[0]+'/'+'*.fit')
flux_dams, spectrum_dams = DP.Preprocessing7('/home/njl/ML/optical/'+kinds[1]+'/'+'*.fit')
flux_db, spectrum_db = DP.Preprocessing7('/home/njl/ML/optical/'+kinds[2]+'/'+'*.fit')

#delate bad data
n = 0
for i in range(len(spectrum_db)):
    if len(spectrum_db[i][0]) < 4096:
        n = i
flux_db.pop(n)
spectrum_db.pop(n)

[array([5282.0195, 5283.235 , 5284.453 , 5285.669 , 5286.8877, 5288.104 ,
        5289.323 , 5290.54  , 5291.7573, 5292.9775, 5294.1953, 5295.416 ,
        5296.6343, 5297.8555, 5299.074 , 5300.2935, 5301.5156, 5302.7354,
        5303.958 , 5305.178 , 5306.4014, 5307.622 , 5308.8433, 5310.0674,
        5311.289 , 5312.5137, 5313.736 , 5314.9614, 5316.184 , 5317.407 ,
        5318.6333, 5319.857 , 5321.0835, 5322.3076, 5323.5347, 5324.76  ,
        5325.9844, 5327.213 , 5328.438 , 5329.667 , 5330.893 , 5332.122 ,
        5333.349 , 5334.5757, 5335.806 , 5337.0337, 5338.264 , 5339.492 ,
        5340.7236, 5341.952 , 5343.181 , 5344.413 , 5345.6426, 5346.8755,
        5348.1055, 5349.339 , 5350.5693, 5351.8003, 5353.034 , 5354.2656,
        5355.5005, 5356.7324, 5357.968 , 5359.2   , 5360.433 , 5361.6694,
        5362.903 , 5364.139 , 5365.3735, 5366.611 , 5367.845 , 5369.08  ,
        5370.3184, 5371.5537, 5372.7925, 5374.0283, 5375.2676, 5376.504 ,
        5377.744 , 5378.981 , 5380.218

In [10]:
#input spectrum
l_list =  flux_cv+flux_dams+flux_db
sl = []
for i in range(len(l_list)):
    sl.append(len(l_list[i]))
sl_min = np.min(sl)
X_train = []
for i in range(len(l_list)):
    X_train.append(l_list[i][0:sl_min])
X_train = np.array(X_train)
size = 4552
#X_train = l_list
X_train = np.stack(X_train)
#X_train = X_train.reshape(len(X_train),size,1) #change the shape to NHWC for CAE input
X_train = X_train.reshape(len(X_train),1,size)#(1214,1,4552)
print(X_train.shape) #print information of training samples

(1214, 1, 4552)


In [12]:
Nepochs = 100 #number of epochs for CAE training
tosavemodel = True #if save the trained CAE model
savename = 'LSTM_reconstruction' #setup if "tosavemodel=True" or "plot_reconstuction=True"

In [18]:
Tprocess0 = time.time()
print('\n', '## DATE PREPARATION RUNTIME:', Tprocess0-Tstart) #Timer

## MAIN ##
#training
H = autoencoder.fit(X_train, X_train,
                    batch_size = 10,
                    epochs=Nepochs,
                    verbose=2,
                    shuffle=True)

Tprocess1 = time.time()
print('\n', '## AE TRAINING RUNTIME:', Tprocess1-Tprocess0) #Timer


 ## DATE PREPARATION RUNTIME: 328.9487843513489
Epoch 1/100


ValueError: in user code:

    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_losses=self.losses)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/

In [18]:
help(autoencoder.fit)

Help on method fit in module tensorflow.python.keras.engine.training:

fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_batch_size=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False) method of tensorflow.python.keras.engine.training.Model instance
    Trains the model for a fixed number of epochs (iterations on a dataset).
    
    Arguments:
        x: Input data. It could be:
          - A Numpy array (or array-like), or a list of arrays
            (in case the model has multiple inputs).
          - A TensorFlow tensor, or a list of tensors
            (in case the model has multiple inputs).
          - A dict mapping input names to the corresponding array/tensors,
            if the model has named inputs.
          - A `tf.data` dataset. Should return a tupl

In [30]:
inputs = tf.random.normal([32, 10, 8])
len(inputs[0][0])

8

In [33]:
len(X_train[0][0])

4552

In [11]:
lstm = tf.keras.layers.LSTM(128,input_shape=(X_train.shape[1:]))

In [12]:
output = lstm(X_train)

In [13]:
output

<tf.Tensor: shape=(1214, 128), dtype=float32, numpy=
array([[ 4.8705737e-25, -3.3902602e-14,  3.5618132e-18, ...,
         2.0434091e-21,  7.6159418e-01,  1.3441982e-24],
       [ 2.7666363e-01, -4.0397489e-01,  2.6729086e-01, ...,
         4.5969221e-03, -6.7452681e-03, -2.3748979e-01],
       [-2.6680392e-01, -3.5334939e-01,  3.0707008e-01, ...,
        -3.6295308e-03, -1.5764056e-02, -1.4165202e-01],
       ...,
       [-3.1333700e-01, -3.3541721e-01,  8.3059445e-02, ...,
         3.9649267e-02,  7.6368069e-03, -3.3683318e-01],
       [ 3.9664745e-01, -3.2815322e-01,  2.5468454e-01, ...,
        -1.1258430e-02,  4.9595172e-03, -5.8493864e-02],
       [ 4.2574435e-01, -1.3672443e-01,  9.1416936e-04, ...,
        -3.8078360e-03, -7.9119764e-03,  1.4533906e-01]], dtype=float32)>